## Aula 6 - Plotagem de um campo de correntes com o basemap.

<h3 style = "color:gray"> importando bibliotecas</h3>

In [1]:
import os
import cmocean
import numpy as np
import datetime as dt
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

<h2 style="color:red">Se você estiver no google colab, por favor descomente as 3 celulas abaixo e depois execute-as, ao finalizar a instalação reinicie o seu colab!.</h2>

In [ ]:
#!pip install basemap

In [ ]:
#!pip install cmocean

In [ ]:
#!pip install netcdf4

<h3 style = "color:blue"> Definindo funções</h3>

In [ ]:
def lon360to180(lon):
    """
    Converts longitude values in the range [0,360]
    to longitude values in the range [-180,+180].
    """
    lon = np.asanyarray(lon)
    
    return ((lon + 180.) % 360.) - 180.

In [ ]:
def hours_to_dates(reference_date, values):
    """"
    Converts hours values in datetime based in a
    specific date.  The reference date must be in the
    format Day/Month/Year Hour:Minutes
    """
    import datetime as dt
    
    date_python = dt.datetime.strptime(reference_date,
                                  '%d/%m/%Y %H:%M')
    
    Time = [date_python + dt.timedelta(hours = x) for x in values]
    
    return Time

<h3 style="color:blue">selecionado o diretorio de trabalho</h3>

In [ ]:
pwd

In [ ]:
os.chdir('/home/gustavo/Documentos/python_aulas/dados')

<h3 style="color:blue"> carregando o dado</h3>

In [ ]:
data = Dataset('dado_aula_06.nc4')

In [ ]:
data.variables

In [ ]:
lat = data['lat'][:]
lon = data['lon'][:]
tempo = data['time'][:]

In [ ]:
lat.min(), lat.max()

In [ ]:
lon.min(), lon.max()

In [ ]:
#transformando o intervalo das longitudes de [0 360] para [-180 180]
lon = lon360to180(lon)

In [ ]:
lon.min(),lon.max()

In [ ]:
condx = np.where((lon>-66) & (lon<-35))
condy= np.where((lat>-10)&(lat<10))

condx = condx[0]
condy = condy[0]

lat2 = lat[condy]
lon2 = lon[condx]

u = data['water_u'][10,0,condy,condx]
v = data['water_v'][10,0,condy,condx]


In [ ]:
# Basemap
m = Basemap(projection='mill',lat_ts=0, \
  llcrnrlon=lon2.min(),urcrnrlon=lon2.max(), \
  llcrnrlat=lat2.min(),urcrnrlat=lat2.max(), \
  resolution='i')


In [ ]:
x = np.transpose(lon2)
y = np.transpose(lat2)

x, y = np.meshgrid(x, y)

r = (u**2 + v**2)**.5

X, Y = m(x, y)

In [ ]:
plt.figure(figsize=(20,8))
#plt.figure(figsize=(16,8))
cmap = m.pcolor(X,Y,r,cmap=cmocean.cm.speed)
cbar = plt.colorbar(cmap)
cbar.set_label('velocidade(m/s)',fontsize=15);
m.drawcoastlines()
m.fillcontinents()
m.drawmapboundary()
m.drawparallels(np.arange(lat.min(),lat.max(),2),labels=[1,0,0,0])
m.drawmeridians(np.arange(lon.min(),lon.max(),2),labels=[0,0,0,1])
#m.drawcountries()
#m.quiver(X[0:99:3,0:87:3], Y[0:99:3,0:87:3], u[0:99:3,0:87:3], v[0:99:3,0:87:3], latlon=False,scale=5)
#plt.colorbar()
#plt.ylabel('Latitude')
#plt.xlabel('Longitude')
plt.title('Corrente Norte do Brasil -10m')
plt.show()

<h3 style="color:blue"> Agora iremos plotar a direção das correntes de todos os tempos do nosso arquivo.</h3>

In [ ]:
# Basemap

#data de referencia
time_ref = dt.datetime.strptime("01/01/2000 00:00",
                                "%d/%m/%Y %H:%M")


# loop para plotar cada imagem
for iterador in range(0,len(tempo),1):
    
    tt = tempo[iterador]
    tempo_dia = time_ref + dt.timedelta(hours=tt)
    tempo_dia = dt.datetime.strftime(tempo_dia,
                                     "%d/%m/%Y %H:%M")
    
    lon = lon360to180(lon)
    
    condx = np.where((lon>-66) & (lon<-35)) #lon min e max
    condy= np.where((lat>-10)&(lat<10)) #lat min e max

    condx = condx[0]
    condy = condy[0]

    lat2 = lat[condy]
    lon2 = lon[condx]

    u = data["water_u"][iterador, 0,condy,condx]
    v = data["water_v"][iterador, 0,condy,condx]
    

    #definindo as propriedades do mapa
    m = Basemap(projection="mill",lat_ts=0, \
      llcrnrlon=lon2.min(),urcrnrlon=lon2.max(), \
      llcrnrlat=lat2.min(),urcrnrlat=lat2.max(), \
      resolution="i")

    
    x = np.transpose(lon2)
    y = np.transpose(lat2)

    x, y = np.meshgrid(x, y)

    r = (u**2 + v**2)**.5

    X, Y = m(x, y)

    #plotando a figura
    plt.figure(figsize=(20,8))
    m.drawcoastlines()
    m.fillcontinents()
    m.drawmapboundary()
    m.drawparallels(np.arange(lat.min(),lat.max(),2),labels=[1,0,0,0])
    m.drawmeridians(np.arange(lon.min(),lon.max(),2),labels=[0,0,0,1])
   
    m.quiver(X[0:499:8, 0:387:8], Y[0:499:8, 0:387:8],
             u[0:499:8, 0:387:8], v[0:499:8, 0:387:8], 
             latlon=False,scale=20)

    
    plt.title('Corrente Norte do Brasil -10m\n'+ tempo_dia)
    plt.show()


